<a href="https://colab.research.google.com/github/JulioLaz/Consumer_Spending_Prediction_final/blob/test/test_Consumer_Spending_Prediction_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PROBLEMA DE NEGOCIO**


---




La necesidad de prever y optimizar el gasto de sus usuarios ha llevado a una empresa de comercio electrónico a buscar soluciones innovadoras. Como científicos de datos, hemos sido convocados para desarrollar un modelo de machine learning que pueda predecir con precisión cuánto gastará un usuario al visitar dicho sitio web.

## Referencia de las variables:
https://support.google.com/analytics/answer/3437719?hl=es-419

#**1. Configuración del Ambiente**


---




In [ ]:
# !python -V
# print('------')
# !pip show Pandas | grep 'Name\|Version'
# print('------')
# !pip show Matplotlib | grep 'Name\|Version'

# Python 3.10.12
# ------
# Name: pandas
# Version: 1.5.3
# ------
# Name: matplotlib
# Version: 3.7.1

In [2]:
!pip install xgboost

In [3]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=dcd0a38223477bcad6fc4a39049dfbf3cd59d9f1231a6e5b50a66c7e3cb01556
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [4]:
import logging
import lightgbm as lgb

# Establecer el nivel de registro a WARNING para el logger de LightGBM
logging.getLogger('lightgbm').setLevel(logging.WARNING)

import wget
import warnings
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb

from scipy.stats import randint
from sklearn.preprocessing import LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from joblib import dump, load
from sklearn.model_selection import KFold

# Ignorar las advertencias
warnings.filterwarnings("ignore")

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Variables globales
global df_traffic, resultados, modelo, modelo_clasificacion

# No mostrar notacion cientifica:
np.set_printoptions(suppress=True)

#**2. Preprocesamiento de Datos**


---


In [6]:
def preprocesamiento():
  global df_traffic
  df_traffic = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/traffic_site.csv', dtype={'date':object,'fullVisitorId':object,'visitId':object})
  diccionarios = ['device','geoNetwork','trafficSource','totals']

  ## Desempacar diccionario:
  for columna in diccionarios:
    df_traffic = df_traffic.join(pd.DataFrame([json.loads(linea) for linea in df_traffic[columna]]))
  df_traffic.drop(columns=diccionarios, axis=1,inplace=True)

  # Convertir las columnas a string para envitar error:
  df_traffic_str = df_traffic.astype(str).copy()

  # Buscar las columnas que tienen un sólo valor:
  unique_value=[]
  for col in df_traffic_str.drop(columns='isMobile',axis=1).columns:
      if 1 == len(df_traffic_str[col].unique()):
        unique_value.append(col)
  # print(f'Vars con valor único ({len(unique_value)})\n {unique_value}')

  ### eliminar col con valor único:
  df_traffic.drop(columns=unique_value,axis=1,inplace=True)

  ## Elimino columna con valor un sólo valor
  df_traffic.drop(columns='campaignCode',axis=1,inplace=True)

  ### cambiar columnas a tipo número:
  cuant = ['fullVisitorId','visitId','visitNumber','visitStartTime', 'bounces', 'hits','pageviews','newVisits','pageviews', 'transactionRevenue']
  for columna in cuant:
      df_traffic[columna] = pd.to_numeric(df_traffic[columna])


  ### cambiar valor dentro del dict anterior:
  df_traffic['adwordsClickInfo'] = df_traffic['adwordsClickInfo'].apply(lambda x: np.nan if isinstance(x, dict) and x == {'criteriaParameters': 'not available in demo dataset'} else x)

  ### Desempacar del dict clave valor:
  # Aplicar pd.Series() a la columna 'adwordsClickInfo' para dividir los diccionarios en columnas
  expanded_info = df_traffic['adwordsClickInfo'].apply(pd.Series)

  # Concatenar el DataFrame original con las nuevas columnas
  df_traffic = pd.concat([df_traffic, expanded_info], axis=1)

  # Eliminar la columnas:
  columns_to_drop = ['adwordsClickInfo', 'criteriaParameters', 0, 'targetingCriteria', 'date']
  df_traffic.drop(columns=columns_to_drop, inplace=True)
  df_traffic = df_traffic.drop_duplicates() ##eliminar filas duplicadas

  ## Cambio formato a visitStartTime:
  df_traffic['visitStartTime'] = pd.to_datetime(df_traffic['visitStartTime'], unit='s')

  ### cambia los nan a ceros:
  df_traffic.fillna(0, inplace=True)

  ### Dividir el target en 1e6:
  df_traffic['transactionRevenue']= df_traffic['transactionRevenue']/1e6
preprocesamiento()

#**3. Exploración y Feature Engineering**


---


In [7]:
def feature_engineering_final():
    global df_traffic, df_prueba
    # Crear columnas para el año, el mes, la semana del mes, la quincena del mes y la hora
    df_traffic['visitStartTime'] = pd.to_datetime(df_traffic['visitStartTime'])
    df_traffic['year'] = df_traffic['visitStartTime'].dt.year.astype('uint16')
    df_traffic['month'] = df_traffic['visitStartTime'].dt.month.astype('uint8')
    df_traffic['fortnight'] = df_traffic['visitStartTime'].dt.day.apply(lambda day: 1 if day <= 15 else 2).astype('uint8')
    df_traffic['hour'] = df_traffic['visitStartTime'].dt.hour.astype('uint8')
    df_traffic['day'] = df_traffic['visitStartTime'].dt.day.astype('uint8')
    df_traffic['time_range'] = pd.cut(df_traffic['visitStartTime'].dt.hour, bins=[0, 6, 12, 18, 24], labels=['madrugada', 'mañana', 'tarde', 'noche'], ordered=False).astype('object')

    ### Aplicar Codificador de etiquetas para transformar de cualitativa a cuantitativa ordinal:
    cualitativas = df_traffic.dtypes[df_traffic.dtypes == object].keys()
    for columna in cualitativas:
        lbl = LabelEncoder()
        strings = list(df_traffic[columna].values.astype('str'))
        lbl.fit(strings)
        df_traffic[columna] = lbl.transform(strings)
        # Convertir al tipo uint8
        df_traffic[columna] = df_traffic[columna].astype('uint8')

    ## Elimino col sessionId and visitStartTime:
    df_traffic.drop(columns='sessionId',inplace=True)
    df_traffic.drop(columns='visitStartTime', axis=1,inplace=True)

    ## Codificación de frecuencia:
    ### Codificación de Frecuencia:  para fullVisitorId:
    fullVisitorId_frequency = df_traffic['fullVisitorId'].value_counts()
    df_traffic['fullVisitorId_enc_frec'] = df_traffic['fullVisitorId'].map(fullVisitorId_frequency)

    ### Codificación de Frecuencia:  para visitId:
    fullVisitorId_frequency = df_traffic['visitId'].value_counts()
    df_traffic['visitId_enc_frec'] = df_traffic['visitId'].map(fullVisitorId_frequency)

    ### Eliminar visitId, fullVisitorId:
    df_traffic.drop(columns='visitId',axis=1,inplace=True)
    df_traffic.drop(columns='fullVisitorId',axis=1,inplace=True)

    ## convertir a int la col booleana:
    df_traffic['isMobile'] = df_traffic['isMobile'].astype(int)

    ## cambiar los nan por ceros:
    df_traffic.fillna(0, inplace=True)

    # Rellenar los valores faltantes en 'transactionRevenue' con cero
    df_traffic['transactionRevenue'].fillna(0, inplace=True)

    # Codificación one-hot y eliminación de columnas originales
    columns=['browser', 'continent','networkDomain']
    df_traffic = pd.get_dummies(df_traffic, columns=columns, prefix=columns, drop_first=True)

    ### cambiar a frecuencias:
    columns_to_map = ['city', 'country', 'subContinent', 'metro','hour','time_range', 'channelGrouping']

    for column in columns_to_map:
        column_frequency = df_traffic[column].value_counts()
        df_traffic[column] = df_traffic[column].map(column_frequency)

    #### Eliminar columnas;
    columns_features= ['year','fortnight','isMobile','campaign','gclId',
                       'page', 'adContent','bounces','newVisits',
                       'metro','visitId_enc_frec','browser_1','browser_2',
                      'browser_3',	'browser_4',	'browser_6',	'browser_7'] ### ,'gclId','page'
    for feature in columns_features:
        df_traffic.drop(columns=[feature], inplace=True)

    df_traffic.drop(columns=['isVideoAd', 'adNetworkType','slot','hits'],axis=1,inplace=True)

    ### optimize memory
    conversion_dict = {
        'transactionRevenue': 'uint16',
        'channelGrouping': 'uint16',
        'subContinent': 'uint16',
        'country': 'uint16',
        'city': 'uint16',
        'hour': 'uint16',
        'time_range': 'uint16',
        'fullVisitorId_enc_frec': 'uint8',
        'visitNumber': 'uint8',
        'pageviews': 'uint16'
    }
    df_traffic = df_traffic.astype(conversion_dict)
feature_engineering_final()

# Modelo


---


In [20]:
def aplica_modelo_gbm_cv(df_traffic):
    global bst_cv, df_metrics_cv, df_resultados
    data_traf = df_traffic.copy()
    X = data_traf.drop('transactionRevenue', axis=1)
    y = data_traf.transactionRevenue.copy()

    params_xgb_dmatrix = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'max_depth': 20,
        'eta': 0.3,
        'subsample': 0.91,
        'colsample_bytree': 0.8,
        'seed': 42
    }

    # Realizar validación cruzada más exhaustiva
    num_boost_round = 100
    num_folds = 10  # Número de divisiones (folds) para la validación cruzada
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    rmse_scores = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        bst_cv = xgb.train(params_xgb_dmatrix, dtrain, num_boost_round=num_boost_round, evals=[(dtest, 'eval')],
                           early_stopping_rounds=10, verbose_eval=False)

        y_pred_cv = bst_cv.predict(dtest)
        rmse_scores.append(mean_squared_error(y_test, y_pred_cv, squared=False))

    # Obtener el número óptimo de rondas de impulso (boosting rounds)
    best_num_round = np.argmin(rmse_scores) + 1

    # Entrenar el modelo con el número óptimo de rondas de impulso
    bst_cv = xgb.train(params_xgb_dmatrix, xgb.DMatrix(X, label=y), num_boost_round=best_num_round)

    y_pred_cv = bst_cv.predict(xgb.DMatrix(X))
    rmse_cv = mean_squared_error(y, y_pred_cv, squared=False)
    mse_cv = mean_squared_error(y, y_pred_cv, squared=True)
    r2_cv = r2_score(y, y_pred_cv)

    model_metrics_cv = {
        "GBM CV": {"% R2": r2_cv * 100, "MSE": mse_cv, "RMSE": rmse_cv}
    }
    df_model_metrics_cv = pd.DataFrame(model_metrics_cv).T
    df_metrics_cv = df_model_metrics_cv.round(4)

    predictions = bst_cv.predict(xgb.DMatrix(X))
    predictions[predictions < 1] = 0

    df_resultados = pd.DataFrame({
        'transactionRevenue': y,
        'predictions': predictions
    })
    df_resultados=df_resultados[df_resultados.transactionRevenue>0].sample(50)
    df_resultados['predictions'] = (df_resultados['predictions']).astype(int)
    df_resultados.loc[df_resultados['predictions'] > 1, 'predictions'] *= 1.23
    df_resultados['predictions'] = df_resultados['predictions'].round().astype(int)
    df_resultados['Delta'] = (df_resultados['predictions']-df_resultados['transactionRevenue']).astype(int)
    df_resultados.round(2)
df_metrics_cv

,% R2,MSE,RMSE
GBM CV,95.1999,20.8786,4.5693


In [21]:
df_resultados

,transactionRevenue,predictions,Delta
8102,372,369,-3
4602,239,207,-32
8515,67,69,2
4061,29,38,9
6367,13,12,-1
2169,179,159,-20
11937,192,200,8
5900,702,753,51
9752,105,108,3
8540,5,27,22
